In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import silhouette_samples
import numpy as np
import matplotlib.pyplot as plt

# Load the Mall Customer Segmentation Data
mall_data = pd.read_csv('Mall_Customers.csv')

# Drop irrelevant columns (CustomerID, Gender) for clustering
# X = mall_data.drop(['CustomerID'], axis=1)
X = mall_data.iloc[:,[3,4]]

# Determine optimal number of clusters using Elbow Method
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

# Plot the Elbow Method graph
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS') # Within cluster sum of squares
plt.show()

# Based on the Elbow Method, we see that the optimal number of clusters is around 5

# Apply K-means clustering with 5 clusters
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(X)

# Get the cluster labels
labels = kmeans.labels_

# Add cluster labels to the original dataset
mall_data['cluster'] = labels

# Calculate silhouette score
silhouette_avg = silhouette_score(X, labels)
print("Silhouette Score:", silhouette_avg)

# Calculate Davies-Bouldin Index
db_index = davies_bouldin_score(X, labels)
print("Davies-Bouldin Index:", db_index)

# Calculate Calinski-Harabasz Index
ch_index = calinski_harabasz_score(X, labels)
print("Calinski-Harabasz Index:", ch_index)

# Visualize the clusters
plt.figure(figsize=(10, 6))

# Plot 1: Scatter plot of features colored by cluster
plt.subplot(1, 2, 1)
plt.scatter(X['Annual Income (k$)'], X['Spending Score (1-100)'], c=labels, cmap='viridis', alpha=0.5)
plt.xlabel('Annual Income (k$)')
plt.ylabel('Spending Score (1-100)')
plt.title('K-means Clustering of Mall Customer Segmentation Data')

# Plot 2: Silhouette plot
plt.subplot(1, 2, 2)
silhouette_values = silhouette_samples(X, labels)
y_lower = 10
for i in range(5):
    cluster_silhouette_values = silhouette_values[labels == i]
    cluster_silhouette_values.sort()
    size_cluster_i = cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i
    color = plt.cm.viridis(float(i) / 5)
    plt.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_silhouette_values, facecolor=color, edgecolor=color, alpha=0.7)
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10

plt.axvline(x=silhouette_avg, color="red", linestyle="--")
plt.yticks([])
plt.xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
plt.xlabel("Silhouette coefficient values")
plt.ylabel("Cluster label")
plt.title("Silhouette plot")
plt.show()